In [2]:
using Revise

In [3]:
using ModifiedSINDy # get the package by following instructions on readme.md from https://github.com/Alz314/ModifiedSINDy

In [4]:
using Agents, GLMakie
using Random
using LinearAlgebra
using LaTeXStrings

In [5]:
using ReinforcementLearning

┌ Info: DataDrivenDiffEq : OccamNet is available.
└ @ DataDrivenDiffEq C:\Users\Alen\.julia\packages\DataDrivenDiffEq\fivVr\src\DataDrivenDiffEq.jl:168


## ABM Implementation

In [6]:
@agent Animal ContinuousAgent{2} begin 
    type::Symbol # :predator or :prey
end

In [7]:
rand_vel(max) = (2 .* map(rand, (Float64, Float64)) .- 1) .* max

function initialize_model(;
    prey_pop = 400,
    pred_pop = 50,
    spacing = 1,
    extent = (200, 200),
    dt = 1,
    sight = 3.0, # predator range of sight
    temperature = 50.0, # temperature is a measure of the average velocity of the animals
    pred_range = 0.5, # predator distance to prey to eat
    pred_reprod = 0.2, # predator reproduction probability
    pred_death = 0.01, # predator death probability
    vel_ratio = 1.25, # ratio of predator to prey velocity 
    prey_reprod = 0.18, # prey reproduction probability
    prey_death = 0.16, # prey death probability
    explore = 0.1, # probability of random exploration
    seed = 42
)
    space2d = ContinuousSpace(extent; spacing = spacing, periodic = true)
    rng = Random.MersenneTwister(seed)

    # make a dict of properties for the model
    properties = Dict([:dt => dt, :sight => sight, :explore => explore, 
                       :temperature => temperature, :pred_range => pred_range, :pred_reprod => pred_reprod, :pred_death => pred_death, 
                       :vel_ratio => vel_ratio, :prey_reprod => prey_reprod, :prey_death => prey_death, :last_update => 0, :update_rate => 100])
    model = ABM(Animal, space2d; rng, properties = properties, scheduler = Schedulers.Randomly())
    prey_vel = temperature / 50
    pred_vel = prey_vel * vel_ratio

    # add agents randomly
    for _ in 1:prey_pop
        add_agent!(model, rand_vel(prey_vel), :prey)
    end
    for _ in 1:pred_pop
        add_agent!(model, rand_vel(pred_vel), :predator)
    end
    return model
end

initialize_model (generic function with 1 method)

In [8]:
function find_closest(target, agents, model, type)
    # find the closest agent to the target
    closest = first(agents)
    closest_dist = euclidean_distance(target, closest, model)
    if closest.type != type
        closest = -1
    end
    for agent in agents
        #skip other predators 
        agent.type != type && continue

        dist = euclidean_distance(target, agent, model)
        if dist < closest_dist
            closest = agent
            closest_dist = dist
        end
    end
    return closest
end

function move_randomly!(agent::Animal, model)
    if rand() < model.properties[:explore]
        if agent.type == :predator
            agent.vel = rand_vel(model.properties[:temperature]*model.properties[:vel_ratio]/50)
        else
            agent.vel = rand_vel(model.properties[:temperature]/50)
        end
    end
    move_agent!(agent, model, model.properties[:dt])
end

get_norm_direction(from, to, model) = Tuple(normalize(collect(get_direction(from.pos, to.pos, model))))

function agent_step!(agent::Animal, model) 
    agent.type == :predator ? predator_step!(agent, model) : prey_step!(agent, model)
end

function predator_step!(agent::Animal, model)
    # get all neighbors within range as an iterator
    neighbors = nearby_agents(agent, model, model.properties[:sight])
    # check if the iterator is empty
    if isempty(neighbors)
        move_randomly!(agent, model)
    else
        # move towards the closest prey
        closest_prey = find_closest(agent, neighbors, model, :prey)
        # if none of the surrounding neighbors are prey, move randomly
        if closest_prey == -1
            move_randomly!(agent, model)
        else
            # update velocity
            pred_vel = model.properties[:temperature]*model.properties[:vel_ratio]/50
            agent.vel = get_norm_direction(agent, closest_prey, model) .* pred_vel
            move_agent!(agent, model, model.properties[:dt])
            # eat the prey
            if euclidean_distance(agent, closest_prey, model) < model.properties[:pred_range]
                remove_agent!(closest_prey, model)
                # reproduce with probability pred_reprod
                if rand() < model.properties[:pred_reprod]
                    add_agent!(agent.pos, model, rand_vel(pred_vel), :predator)
                end
            end
        end
    end
    # die with probability pred_death
    if rand() < model.properties[:pred_death]
        remove_agent!(agent, model)
    end
end

function prey_step!(agent::Animal, model)
    # get all neighbors within range as an iterator
    neighbors = nearby_agents(agent, model, model.properties[:sight])
    # check if the iterator is empty
    if isempty(neighbors)
        move_randomly!(agent, model)
    else
        # move away from the closest predator
        closest_pred = find_closest(agent, neighbors, model, :predator)
        # if none of the surrounding neighbors are predators, move randomly
        if closest_pred == -1
            move_randomly!(agent, model)
            return
        end
        #if the predator is at the exact same position as the prey, move randomly
        if closest_pred.pos == agent.pos
            move_randomly!(agent, model)
        else
            # update velocity
            agent.vel = get_norm_direction(closest_pred, agent, model) .* model.properties[:temperature]./50
            move_agent!(agent, model, model.properties[:dt])
        end
    end
    # reproduce with probability prey_reprod
    if rand() < model.properties[:prey_reprod]
        add_agent!(agent.pos, model, rand_vel(model.properties[:temperature]/50), :prey)
    end
    # die with probability prey_death
    if rand() < model.properties[:prey_death]
        remove_agent!(agent, model)
    end
end

function system_update!(model)
    # update the last_update property
    model.properties[:last_update] += 1
    # update the model every update_rate steps
    if model.properties[:last_update] >= model.properties[:update_rate]
        # reset the last_update property
        model.properties[:last_update] = 0
        # choose a new random pred reproduction probability
        #model.properties[:pred_reprod] = rand(0.005:0.005:0.5)
    end
end

step!(model::ABM, n::Int = 1) = Agents.step!(model, agent_step!, system_update!, n)

step! (generic function with 2 methods)

## Running and visualizing the ABM

In [9]:
# need to run for the plots to display properly
ac(a::Animal) = a.type == :predator ? :red : :green

ac (generic function with 1 method)

The following is how to run a single simulation and get a dataframe, skipping every 10 steps

In [10]:
model = initialize_model()
skip(model, s) = s % 10 == 0 # you can change this to skip more or less often
steps = 2000
data = run!(model, agent_step!, steps; adata = adata, when = skip)

UndefVarError: UndefVarError: `adata` not defined

This should open a window where you can play with different parameters for the simulation

In [11]:
params = Dict(
    :temperature => 1:1:100,
    :vel_ratio => 0.1:0.1:3.0,
    :sight => 0.2:0.2:10.0,
    :pred_range => 0.1:0.1:5.0,
    :pred_reprod => 0.01:0.01:0.5,
    :pred_death => 0.01:0.01:0.5,
    :prey_reprod => 0.01:0.01:0.5,
    :prey_death => 0.01:0.01:0.5,
    :explore => 0.01:0.01:1
)
model = initialize_model()
fig, ax, abmobs = abmplot(model; ac = ac, agent_step! = agent_step!, model_step! = Agents.dummystep, params)
fig

This saves a video of a single simulation

In [12]:
model = initialize_model()
abmvideo("pred_prey.mp4", model, agent_step!, dummystep; frames = 200, ac = ac)

This should generate plots of each agents population in real time with the simulation.  
Unfortunately, stopped working after the last Agents.jl update for some reason

In [13]:
ispred(a::Animal) = a.type == :predator
isprey(a::Animal) = a.type == :prey
adata = [(ispred, count), (isprey, count)]
model = initialize_model()
fig, abmobs = abmexploration(model;
    agent_step! = agent_step!, model_step! = Agents.dummystep, params, ac = ac,
    adata, alabels = ["Predator Count", "Prey Count"]
)
fig

KeyError: KeyError: key :stepclick not found

## Preprocessing Data

How I am normalizing the data

In [14]:
function normalize_data(data::AbstractMatrix)
    # normalize data to be between 0 and 1; dramatically improves performance
    data_norm = zeros(size(data))
    for i in 1:size(data, 2)
        data_norm[:, i] = data[:, i]./maximum(data[:, i])
    end
    return data_norm
end

normalize_data (generic function with 1 method)

In [37]:
model = initialize_model()
skip(model, s) = s % 10 == 0
agents_data = run!(model, agent_step!, 2000; adata = adata)
# make a matrix from the count_ispred and count_isprey columns of the agents_data dataframe
dataset = hcat(agents_data[1].count_isprey, agents_data[1].count_ispred)
# cutoff last 10 percent of data
dataset = dataset[1:floor(Int, size(dataset, 1)*0.9), :]
norm_data = normalize_data(dataset)
dt = 1
ues, dues = SG_smoothing_optim(norm_data, dt; loss_function = 2, disp_loss_landscape = false);

This simply plots the smoothed data from the previous block

In [31]:
function plot_population_2d(data::AbstractMatrix; title = "Data")
    # set up figure
    figure = Figure(resolution = (600, 400))
    ax = figure[1, 1] = Axis(figure; xlabel = "Step", ylabel = "Population", title = title)
    # plot individual populations from data
    herbivorel = lines!(ax, data[:, 2], color = :cornsilk4)
    grassl = lines!(ax, data[:, 1], color = :green)
    # add legend
    figure[1, 2] = Legend(figure, [grassl, herbivorel], ["Prey", "Predator"])
    display(figure)
end

plot_population_2d(ues)

GLMakie.Screen(...)

## Interactive SINDy

Utility functions

In [17]:
# function to get the nearest odd number that is less than the given number
function nearest_odd(n::Real)
    # first convert n to an integer
    n = round(Int, n)
    if n % 2 == 0
        return n - 1
    else
        return n - 2
    end
end


function build_eqn(m, basis_latex)
    # build the equation strings using matrix m and basis_latex, ignoring terms with coefficient of 0
    eqn1 = "\\frac{dx}{dt}="
    eqn2 = "\\frac{dy}{dt}="
    eqn1_first = true
    eqn2_first = true
    for i in 1:size(m, 1)
        if m[i, 1] != 0
            if eqn1_first || m[i, 1] < 0
                eqn1 *= string(round(m[i,1], sigdigits = 2))*basis_latex[i]
                eqn1_first = false
            else
                eqn1 *= "+"*string(round(m[i,1], sigdigits = 2))*basis_latex[i]
            end
        end
        if m[i, 2] != 0
            if eqn2_first || m[i, 2] < 0
                eqn2 *= string(round(m[i,2], sigdigits = 2))*basis_latex[i]
                eqn2_first = false
            else
                eqn2 *= "+"*string(round(m[i,2], sigdigits = 2))*basis_latex[i]
            end
        end
    end 
    return latexstring(eqn1), latexstring(eqn2)
end

build_eqn (generic function with 1 method)

The following code gives you a slider on window sizes for Savitzky Golay smoothing to see the effects in real time

In [18]:
function view_windowsize_effect(data::AbstractMatrix)
    # set up figure
    fig = Figure(resolution = (600, 400))
    ax = fig[1, 1] = Axis(fig; xlabel = "Step", ylabel = "Population", title = "Window Size Effect")
    # get initial best estimate of smoothing
    ues, dues, w_o = SG_smoothing_optim(data, 1; loss_function = 2, disp_loss_landscape = false, ret_window = true)
    ues = Float32.(ues)
    # set up the smoothing slider
    windowsize = Slider(fig[2, 1], range = 3:2:299, startvalue = w_o)

    # set up the plot
    prey_data = Observable{AbstractArray}(ues[:, 1])
    pred_data = Observable{AbstractArray}(ues[:, 2])
    #disp_data = (x = prey_data, y = pred_data)
    # set up action for when the slider is changed
    w = windowsize.value
    function smoothing_wrapper(w)
        ues, dues = SG_smoothing(data, w, 1)
        ues = Float32.(ues)
        return ues
    end
    prey_data = @lift(smoothing_wrapper($w)[:, 1])
    pred_data = @lift(smoothing_wrapper($w)[:, 2])
    # plot the data
    #preyl = lines!(ax, to_value(disp_data.x), color = :green)
    #predl = lines!(ax, to_value(disp_data.y), color = :cornsilk4)
    preyl = lines!(ax, prey_data, color = :green)
    predl = lines!(ax, pred_data, color = :cornsilk4)
    # add legend
    fig[1, 2] = Legend(fig, [preyl, predl], ["Prey", "Predator"])
    display(fig)
end

view_windowsize_effect(norm_data)

GLMakie.Screen(...)

The following gives you a huge window to not only run different simulations, but also smooth them and run SINDy.  
Note: The first time you run SINDy, it will take a lot of time precompiling the code

In [35]:
# set up figure
fig = Figure(resolution = (1444, 980))
data = Observable(norm_data)

ax = Axis(fig[1:2, 1]; xlabel = "Step", ylabel = "Population", title = "Smoothed Data")
# get initial best estimate of smoothing
ues, dues, w_o = SG_smoothing_optim(data[], 1; loss_function = 2, disp_loss_landscape = false, ret_window = true, order = 3)
#ues = Float32.(ues)
# set up data generation sliders 
data_sliders = SliderGrid(
    fig[1, 3],
    (label = "Timespan:", range = 100:50:6000, startvalue = 2000),
    (label = "Record every N steps:", range = 1:50, startvalue = 1),
    (label = "Initial Prey Population:", range = 50:50:1000, startvalue = 400),
    (label = "Initial Predator Population:", range = 10:10:500, startvalue = 50),
    (label = "Temperature:", range = 1.0:1.0:100.0, startvalue = 50.0),
    tellheight = false)
n_steps = data_sliders.sliders[1].value
record_every = data_sliders.sliders[2].value
prey_pop = data_sliders.sliders[3].value
pred_pop = data_sliders.sliders[4].value
temp = data_sliders.sliders[5].value
# set up the smoothing sliders
sg_sliders = SliderGrid(
    fig[3, 3],
    (label = "Window Size", range = 5:2:301, startvalue = w_o),
    (label = "Order", range = 2:10, startvalue = 3),
    (label = "Ensemble Batches", range = 0:10:1000, startvalue = 100),
    (label = "Train-Test Split", range = 0:0.05:1.0, startvalue = 0.9),
    (label = "Ensemble Tolerance", range = 0:0.01:1.0, startvalue = 0.9),
    tellheight = false)
w = sg_sliders.sliders[1].value
o = sg_sliders.sliders[2].value
batches = sg_sliders.sliders[3].value
train_test_split = sg_sliders.sliders[4].value
tol = sg_sliders.sliders[5].value
# set up button to generate data
data_btn = Button(fig[2, 3], label = "Generate Data", tellwidth = false, tellheight = false)
# set up button to run SINDy
sindy_btn = Button(fig[4, 3], label = "Run SINDy", tellwidth = false, tellheight = false)
# set up SINDy output text
eqn1 = Observable(L"\frac{dx}{dt} = \alpha x - \beta x y")
eqn2 = Observable(L"\frac{dy}{dt} = \delta x y - \gamma y")
txt = Observable("Expected equations:")
# set up the plot
prey_data = Observable{AbstractArray}(ues[:, 1])
pred_data = Observable{AbstractArray}(ues[:, 2])
prey_deriv_data = Observable{AbstractArray}(dues[:, 1])
pred_deriv_data = Observable{AbstractArray}(dues[:, 2])
# set up action for when the slider is changed
function smoothing_wrapper(w, o)
    ues, dues = SG_smoothing(data[], w, 1; order = o)
    #ues = Float32.(ues)
    return ues
end

function smoothing_wrapper_deriv(w, o)
    ues, dues = SG_smoothing(data[], w, 1; order = o)
    #dues = Float32.(dues)
    return dues
end

prey_data = @lift(smoothing_wrapper($w, $o)[:, 1])
pred_data = @lift(smoothing_wrapper($w, $o)[:, 2])
prey_deriv_data = @lift(smoothing_wrapper_deriv($w, $o)[:, 1])
pred_deriv_data = @lift(smoothing_wrapper_deriv($w, $o)[:, 2])
# set up action for when data generation button is pressed
on(data_btn.clicks) do _
    model = initialize_model(prey_pop = prey_pop[], pred_pop = pred_pop[], temperature = temp[])
    skip(model, s) = s % record_every[] == 0
    agents_data = run!(model, agent_step!, n_steps[]; adata = adata, when = skip)
    # make a matrix from the count_ispred and count_isprey columns of the agents_data dataframe
    dataset = hcat(agents_data[1].count_isprey, agents_data[1].count_ispred)
    # cutoff last 10 percent of data
    dataset = dataset[1:floor(Int, size(dataset, 1)*0.9), :]
    data.val = normalize_data(dataset)
    ues, dues = SG_smoothing(data[], w[], 1; order = o[])
    prey_data.val = ues[:, 1]
    pred_data.val = ues[:, 2]
    prey_deriv_data.val = dues[:, 1]
    pred_deriv_data.val = dues[:, 2]
    w[] = w.val
end
# set up action for when the button is pressed
on(sindy_btn.clicks) do _
    # remake ues and dues with the data we have
    ues = [prey_data[] pred_data[]]
    dues = [prey_deriv_data[] pred_deriv_data[]]
    basis = [
        BasisTerm(u -> u[:, 1]),
        BasisTerm(u -> u[:, 2]),
        BasisTerm(u -> u[:, 1] .* u[:, 2]),
        BasisTerm(u -> u[:, 1] .^ 2),
        BasisTerm(u -> u[:, 2] .^ 2)
    ]
    iter = 10
    cs = exp10.(-10:0.1:-1)

    alg = PFA(cs, 0.8)
    prob = SINDy_Problem(ues, dues, 1, basis, iter, alg; STRRidge = true, )
    m, _ = ensemble_solve_SINDy(prob,batches[], train_test_split[], tol[], true)

    # get the equations
    basis_latex = [
        "x",
        "y",
        "xy",
        "x^2",
        "y^2"
    ]
    new_eqn1, new_eqn2 = build_eqn(m, basis_latex)
    
    # update the text
    txt[] = "SINDy Model:"
    eqn1[] = new_eqn1
    eqn2[] = new_eqn2
end
# plot the data
preyl = lines!(ax, prey_data, color = :green)
predl = lines!(ax, pred_data, color = :cornsilk4)
split_pos = @lift(floor(Int, size(data[])[1]*$train_test_split))
splitl = vlines!(ax, split_pos, color = :orange, linestyle = :dash)

ax_deriv = Axis(fig[1:2, 2]; xlabel = "Step", ylabel = "Derivative", title = "Estimated Derivatives")
prey_derivl = lines!(prey_deriv_data, color = :green)
pred_derivl = lines!(pred_deriv_data, color = :cornsilk4)
splitl = vlines!(ax_deriv, split_pos, color = :orange, linestyle = :dash)

# plot phase space
ax_phase = Axis(fig[3:4, 1]; xlabel = "Prey Population", ylabel = "Predator Population", title = "Phase Space")
lines!(ax_phase, prey_data, pred_data, color = :black)
# plot estimated noise
ax_noise = Axis(fig[3:4, 2]; xlabel = "Step", ylabel = "Noise", title = "Estimated Noise")
prey_noise = @lift(data[][:, 1] .- $prey_data )
prey_noisel = lines!(prey_noise, color = :green)
pred_noise = @lift((data[][:, 2] .- $pred_data ))
pred_noisel = lines!(pred_noise, color = :cornsilk4)
Label(fig[5, 3], txt, justification = :center, tellheight = true, tellwidth = false, height = 0.1)
Label(fig[6, 3], eqn1, justification = :center, tellheight = true, tellwidth = false, height = 15)
Label(fig[7, 3], eqn2, justification = :center, tellheight = true, tellwidth = false, height = 15)
# add legend
fig[5, 1:2] = Legend(fig, [preyl, predl], ["Prey", "Predator"], orientation = :horizontal)
display(fig)

GLMakie.Screen(...)

Now running SINDy the normal (boring) way

In [20]:
basis = [
    BasisTerm(u -> u[:, 1]),
    BasisTerm(u -> u[:, 2]),
    BasisTerm(u -> u[:, 1] .* u[:, 2]),
    BasisTerm(u -> u[:, 1] .^ 2),
    BasisTerm(u -> u[:, 2] .^ 2)
]
print("Expected coefficients:")
expected_ips = [1 0; 0 1; 1 1; 0 0; 0 0]

Expected coefficients:

5×2 Matrix{Int64}:
 1  0
 0  1
 1  1
 0  0
 0  0

In [38]:
iter = 10
cs = exp10.(-10:0.1:-1)
tspan = (0, 1000-5)
batches = 100; pct_size = 0.9; tol = 0.9; parallel = true

alg = PFA(cs, 0.8)
prob = SINDy_Problem(ues, dues, dt, basis, iter, alg; STRRidge = true, )
m, ips = ensemble_solve_SINDy(prob,batches, pct_size, tol, parallel)
display(m) # returned matrix
display(ips) # inclusion probability

5×2 Matrix{AbstractFloat}:
  0.00198013   0.0
  0.0         -0.00497864
 -0.00488867   0.0268356
  0.0          0.0
  0.0          0.0

5×2 Matrix{Float64}:
 1.0   0.0
 0.0   1.0
 1.0   1.0
 0.64  0.0
 0.0   0.67

## Reinforcement learning (Unfinished)

In [25]:
Base.@kwdef mutable struct ABMEnv{T<:AbstractFloat} <: AbstractEnv
    reward::Union{Nothing, Int} = nothing
    model::AgentBasedModel
    state::Vector{T} # state is a vector of the predator and prey counts and rate of change and the current growth rate
    action_space::Space{Vector{ClosedInterval{T}}}
    temperature::T
    done::Bool
    tolerance::T
end

function RLBase.reset!(env::ABMEnv)
    env.model = initialize_model()
    env.growth_rate = env.model[:temperature]
    env.done = false
    env.state = [0.0, 0.0, 0.0, 0.0, 0.0]
end

function RLBase.is_terminated(env::ABMEnv)
    if env.model.properties[:step] >= 10000
        return true
    elseif env.model.properties[:prey] <= 0.0 || env.model.properties[:predator] <= 0.0
        return true
    end
    return false
end

function RLBase.state(env::ABMEnv)
    return env.state
end

function RLBase.action_space(env::ABMEnv)
    return env.action_space
end

function RLBase.reward(env::ABMEnv)
    if env.model.properties[:prey] <= 0.0 || env.model.properties[:predator] <= 0.0
        return -1
    else
        ratio = env.model.properties[:prey] / env.model.properties[:predator]
        if abs(ratio - 2.0) > env.tolerance
            return -1
        else
            return abs(ratio)
        end
    end
end

function (env::ABMEnv)(action)
    env.growth_rate = max(min(env.temperature + env.actions[action], 100.0), 1.0)
    env.model.properties[:temperature] = env.temperature
end